## < CNN 심화 과제2 – 논문 리뷰 >
### 논문 : YOLO(you only look once) - unified, real-time object detection

[Abstract]
기존의 detector -> classifier problem이 detection 기능을 하도록 한다.
but YOLO는 regression problem
– 기대되는 목적값이 continuous 한 연속성을 가지고 있는 문제로 바라보자
- 목적값 : spatially separated bounding boxes (박스 center relative coordinates 및 relative width, height and associated class probabilities – 총 5가지 value

[Single network] optimization에서 boxing과 class probability가 동시에 일어난다
- can be directly optimized 

[Good performance]
- YOLO : 45 frames per seconds
- Fast YOLO : 155 frames per seconds
- real-time process possible
- state-of-the-art detection system에 비해 more box localization error but less likely to predict false positive on background
- YOLO learn very general representations of objects
- 인공 이미지나 자연 이미지나 다 유도리있게 맞아들어간다

[Introduction]
- 기존 : boxing by sliding window approach, pre or post processing by cv or etc...
- separately trained boxing and classification
- 그래서 slow and hard to optimize
- yolo : simultaneous predict box and class by unified model 여기에 착안헤서 이름이 you only look once가 되었다
unified model로 인한 기존 model에 비해 개선 된 점
1) YOLO is extremely fast
2) YOLO reasons globally about the image when making predictions
- so it implicitly encodes contextual information about classes as well as their appearance
3) YOLO learns generalizable representations of objects
- YOLO still lags behind state-of-the-art in accuracy
- tradeoffs : quickly identify objects , precisely localize some objects

[Unified Detection]

(Step 1)
- Divide input image into SxS grid
- If the center of an object falls into a grid cell -> that grid cell is responsible for detecting that object
 
(Step 2)
- Each grid cell predicts B(bounding boxes) and confidence scores for those boxes
- confidence score : that the box contains an object + how accurate it thinks the box is that it predicts
- confidence score = Pr(Object)*IOU(truth,pred)
- IOU는 prediciton과 truth box의 공통부분 면적/truth box+pred box 총 면적
- Pr(Object) pred-box내에 object(class 관계없이)가 있는가의 확률
- 해당 cell에 object가 있으면(label box의 center point) Pr(Object)=1
- 해당 cell에 object가 없으면(label box의 center point) Pr(Object)=0
- Each bounding box : 5 predictions (x,y,w,h,confidence)
- (x,y) coordinates represent the center of the box relative to the bounds of the grid cell
- (w,h) represents the width and height predicted relative to the whole image
- Each grid cell also predict C conditional class probabilities Pr(Class_i|Object)
- 따라서 only predict on set of class probabilities per grid cell, regardless of the number of boxes B (S를 큰 값으로 잡아야만한다 -> 모옵시 많고 작은 물체들이 있을 경우 단점으로 작용한다고 생각된다)
- C*box_confidence = Pr(Class_i|Object)*confidence score
- C*box_confidence = Pr(Class_i|Object)*Pr(Object)*IOU(truth,pred)
- C*box_confidence = Pr(Class_i)*IOU(truth,pred) ; class-specific confidence score
- class-specific confidence score : encode both the probability of that class appearing in the box and how well the predicted box fits the object
- tensor 수 : S*S*(B*5+C)

[Network Design]
- Inspired by GoogleNet model : Inception module, not auxiliary classifier
- 24 convolution layer + 2 fully connected layer
- GoogleNet’s Inception module -> 1x1 reduction layer followed by 3x3 convolution layers (solution for nonlinear)
- 1x1 reduction layer의 nonlinear 해결성 보여보기(2d plot에서)
- Fast YOLO : fewer convolutional layers (9 instead of 24 layers) 그 외에는 same

[Training]
- Detection often requires fine-grained convolutional layers -> 224x224 to 448x448 input
- Pretrained 20 conv layers (끝단만 그냥 softmax처리 잘하면 되니까) 사용
- 이후 4 conv layers와 2 fully connected layers 적용해서 위의 20conv는 weight 기존 pretraining 했던 거 사용하고, 나머지는 weight initialization한다
- Using a linear activation function for the final layer and all other layers use the leaky rectified linear activation
- Optimize the sum-squared error : because it is easy to optimize
- 대부분의 grid는 object를 포함하지 않는 경우가 많다 -> boxing보다 object의 유무에만 맞게 back-propagation이 일어날 가능성이 크다 -> predicting bounding box coordinates에는 가점을 no-object detection에서의 error에서는 감점을 어느 정도 하여 object 유무에만 fitting되는 것이 아닌 object 유무와 bounding box에도 fitting이 되도록 하였다
- =5, =0.5
- boxing을 할 때는 작게 해서 놓치는 거 보다 크게 해서 물체가 박스안에 들어가는게 좋으므로 error 구할 때 width랑 height는 root를 씌운다
- boxing 테두리의 경우에는 B개로 우리가 사전에 정해놓은 값의 개수만큼 각 grid cell에서 boxing이 이루어 진다
- 이때 우리는 가장 정확한 boxing 하나만 사용하면 되므로 각 grid에서 학습 후 IOU가 제일 높은 boxing model을 선택하여 개를 통해 차후에 object의 class를 추정한다 (하나의 boxing model이 채택되면 나머지 boxing model들은 그 grid에서는 들러리다 그냥)
  (안써~)
- 마직막으로 loss function에서 cell i, bounding box j 에 따라 구분하게 하는 인자가 존재한다
- 딥러닝에서 loss function이 개인적인 생각으로 중요성의 반 이상을 차지한다 생각하여 끝에 개인적인 해석을 추가하였다 (수식이 너무많아 사진으로 첨부하였습니다)
- 논문에서는 PASCAL VOC2007, VOC2012 데이터에 대하여 batch, epoch, drop-out, decay rate, learning rate를 조절하며 학습을 진행한 것을 얘기하고 있는데, 이는 학습하는 데이터의 성향과 특성에 따라 그때 그때 조절해야하는 hyper-parameter라 생각하므로 논문에 나와있는게 정답이라고 생각하지 않아 참고정도만 하였습니다
- data의 성향을 파악하고 위의 hyper-parameter를 조절하며 필요에 따라 추가 layer를 첨부하며 해당 data set 특성에 가장 overfitting되는(training set에 overfitting되는 것이 아닌 data의 성향에 overfitting된다는 의미) network을 만들어 내는게 data analysis의 진짜 실력이지 않을까 하는 생각을 잠시 해보았습니다
- (아마 이런 능력은 많은 data를 분석해보고 training해보는 경험에서 나올 것 같습니다)

[Inference]
- bounding box와 classifier가 동시에 일어나 매우매우 fast하다
- 또 S*S grid로 나누었는데 일반적으로 한 grid에 하나의 object가 떨어져 detection과 boxing이 잘 이루어 지지만, 매우 큰 object 혹은 grid cell의 경계에 object boxing center가 잡히는 경우 여러 개의 cell에서 bounding box를 생성하는 문제점이 생긴다
- 그래서 non-maximal suppression으로 문제를 어느정도 잡아낸다
(reference : https://dyndy.tistory.com/275)
- non-maximal suppression 과정
- 1. 동일한 클래스에 대해 높은-낮은 confidence 순서로 정렬한다.
- 2. 가장 confidence가 높은 boundingbox와 IOU가 일정 이상인 boundingbox는 동일한 물체를 detect했다고 판단하여 지운다. 보통 50%(0.5)이상인 경우 지우는 경우를 종종 보았다.
- 3. 위의 과정을 confidence 내림차순 나열에서 하나씩 내려가며 실행하며 지워나간다

[Limitations of YOLO]
- 우선 grid당 classifier가 1개밖에 안되기 때문에 작은 물체의 경우 가까이 모여있으면 전부 detect하기가 어렵다
- 또한, model이 train data들을 통해 학습하기 때문에 새롭거나 unusual aspect에 general하게 적용되는데 좀 어려움이 있었다 (그래도 다른 detection 모델들과 비교하면 꽤 general 하게 적용된다) : 어차피 layer에서 downsampling을 하게되므로 coarse한 feature를 잡는다
- 마지막으로 box가 커서 object를 안에 담으면 괜찮은데 작아서 못 담으면 IOU에 더 큰 영향을 미치므로 루트를 씌워 어느 정도 대안은 취하였지만, 그래도 아직 완전히 해결이 되지는 않는 것으로 보인다 (왜 작으면 더 큰 영향을 IOU에 미치는지 끝에 사진 첨부)
  주된 error가 incorrect localization이다

[YOLO VS other models]
1. DPM(deformable parts models) R-CNN등등 보다 더 좋은 accuracy 더 빠른 time rate을 가진다
2. real time의 경우 fast yolo가 155FPS나온다 (다른 model들은 20 강제로 키운 DPM의 경우 100이 한계 – 같은 computing 환경)
3. YOLO는 background error가 fast-RCNN에 비해 덜 생긴다 -> 합치자
   yolo랑 fast-rcnn각각 돌리고 박스 생긴거에서 공통부분 IOU로 다시 예측하면 성능 향상
   그렇다고 다른 모델과 합치면 큰 변화 없거나 떨어짐 즉 fast-rcnn과 yolo의 error가 상     이한 성향을 띠므로 같이 사용시 상대의 error를 더 보완해줌
   서로 왜 상이한 error를 띠는지는 안나와 있음 (고민해봐도 잘 모르겠습니다)
   하지만 computation time은 fast-rcnn이 압도적으로 높아서 결국 yolo는 미리 계산하고     fast-rcnn이 계산끝나기 기다렸다가 맞춰보고 이런식으로 진행되서 time rate은             fast-rcnn이 된다
   하지만 성능이 높아지니까 굳굳~
- 여기까지가 논문의 순수 내용, 아랫부분은 제 해석과 나름의 증명 참고 논문 관련 내용입니다

[Appendix]
- 논문 제목 : object detection networks on convolutional feature maps
- pretrained 된 model의 weight을 초기값으로 학습을 하였을 때 성능의 개선이 있다고하여 해당 논문을 찾아보았습니다(논문 첨부도 같이 하였습니다)
- yolo의 경우 24conv-layers중 20개를 pretrained하여 사용합니다
- region-wise classifier architecture 대상으로 region-dependent convolutional features들만 해당이 됩니다(yolo의 경우 S*S로 grid를 나누고 학습을 진행하니 region dependent합니다)
- network를 보는 또다른 insight를 제공한다는데에 의의가 있는 것으로 보입니다
- detection 알고리즘에서 boxing 네트워크는 제외하고 feature map을 통한 classifier network에만 적용되며 vgg, googlenet, resnet, rcnn등의 네트워크에서 pre-trained weight을 이용하여 학습을 시켰더니 더 좋더라가 결론인 것 같습니다
- 구체적인 수식이나 고찰, 도출등이 있기를 바랬는데 그냥 해보니 그렇더라 의 느낌이었습니다